In [1]:
import cv2,os,glob,csv,time
import numpy as np
import pyqtgraph as qt
import pickle
import pdb

global color, upperPt, lowerPt, frameCenter, refPt, color_data, scaled, frame,diameter
upperPt = []
lowerPt = []
frameCenter = []
refPt = []
scaled = False
diameter = []

color_data = {
'red_pos': [],\
'red_min': [],\
'red_max': [],\
'yellow_pos': [],\
'yellow_min': [],\
'yellow_max': [],\
'lowerPt': [],\
'upperPt': [],\
'frameCenter': [],\
'refPt': [],\
'diameter': [],\
}

color_set = ['red','yellow']

In [2]:
def histNorm(frame,plotting=False):	
	"""this function does some shit
	Frame must be cropped (tracking area only)"""

	hist, bins = np.histogram(frame.flatten(),256,[0,256])
	
	cdf = hist.cumsum()
	cdf_normalized = cdf * hist.max()/cdf.max()
	#TODO: save cdf from example frame to make normalized hist for subsequent frames
	
	if plotting:
		plt.plot(cdf_normalized, color = 'b')
		plt.hist(frame.flatten(),256,[0,256])
		plt.xlim([0,256])
		plt.legend(('cdf','histogram'), loc = 'upper left')
		plt.show()
		
		#TODO: delete image with keystroke
	
	#mask and scale histogram
	cdf_m = np.ma.masked_equal(cdf,0)
	cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
	cdf = np.ma.filled(cdf_m,0).astype('uint8')
	
	new_image = cdf[frame]
	
	return new_image	

def get_center(event,x,y,flags,param):			
	"""Finds center of an image using mouse click"""
	global frameCenter, frame	
	if event == cv2.EVENT_LBUTTONDOWN:
		frameCenter = (x,y)
		cv2.circle(frame,(x,y),10,(0,0,0),-1)
		cv2.imshow('Click on center', frame)
		print('Center is '+ str(frameCenter))
		cv2.destroyAllWindows()
		
		color_data["frameCenter"] = frameCenter
        
def get_circ(event,x,y,flags,param):
	"""Finds edge of drum for distance estimations"""	
	global frame, frameCenter, diameter
	diameter = []
	if event == cv2.EVENT_LBUTTONDOWN:
		#draw circles where user clicks
		circPt = (x,y)
		cv2.circle(frame,(x,y),1,(0,0,0),1)
		cv2.destroyWindow('Click 6 points on drum')
		cv2.imshow('Click 6 points on drum', frame)
		diameter.extend(np.sqrt((frameCenter[0]-list(x))**2 + (frameCenter[1]-list(y))**2))
		color_data['diameter'] = diameter
		print color_data['diameter'] 
#	return diameter

def get_color(event,x,y,flags,param):
	"""gets location of color from button click, derives color value parameters for tracking"""
	global color_loc,setColor,setColorMax, setColorMin,frame	#define these
	color_loc = []	
	setColorMin = []
	setColorMax = []	
#	frame = histNorm(frame)
	hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
	
	if event == cv2.EVENT_LBUTTONDOWN:		#check 
		color_loc = (x,y)		
		setColor = hsv[y-2:y+2,x-2:x+2,:]
		setColor = np.uint8(setColor)		
		setColorMin = (np.min(setColor[:,:,0]),np.min(setColor[:,:,1]),np.min(setColor[:,:,2]))	 	#not sure this will gake min of each column
		setColorMax = (np.max(setColor[:,:,0]),np.max(setColor[:,:,1]),np.max(setColor[ :,:,2]))
		print hsv[color_loc[0],color_loc[1],:]
		print color_loc
	
		#export data
		color_data[color+'_pos']=color_loc
		color_data[color+'_min']=setColorMin
		color_data[color+'_max']=setColorMax
		
		cv2.destroyAllWindows()
		
def get_reference(event, x,y,flags, param):
	"""Selects ROI (100 x 100 rectangle) on frame from which the grating rotation speed can be calculated.
	ROI is a rectangle whos bounds are determined by clicking the position of the upper
	left corner of the rectangle. Image window name must be 'image'. """
	global refPt,frame
	if event == cv2.EVENT_LBUTTONDOWN:
		refPt = [(x, y)]
		refPt.append((x+100,y+100))
		#accessing the values within the rectange would use: image[refPt[0][0:1],refPt[1][0:1]]
		cv2.rectangle(frame,refPt[0],refPt[1],(255,255,0),2)
		cv2.imshow('Reference region made',frame)
		cv2.destroyAllWindows()
		
		color_data["refPt"] = refPt
		
def resize_frame(event,x,y,flags,param):
	"""Click and drag to select entire probable search area starting in upper left corner."""		
	global upperPt, lowerPt, frame, color_data
	if event == cv2.EVENT_LBUTTONDOWN:
		upperPt = [x,y]
		#upperPt = list(upperPt[0])
		print ('Upper pt is{}'.format(upperPt))
	if event == cv2.EVENT_LBUTTONUP:
		lowerPt = [x,y]
		#lowerPt = list(lowerPt[0])
		print ('Lower pt is {}'.format(lowerPt))
# 		cv2.rectangle(frame, upperPt[0], lowerPt[0],(0,0,0),1)
		cv2.destroyWindow('Select region of interest')			
		#crop frame
		frame = frame[upperPt[1]:lowerPt[1],upperPt[0]:lowerPt[0]]
		cv2.imwrite('resized.jpg',frame)
		frame = histNorm(frame)
		print('Resize successful')
		cv2.imshow('Select region of interest', frame)	

		color_data["upperPt"] = upperPt
		color_data["lowerPt"] = lowerPt
		cv2.destroyAllWindows()
        
def import_globals():
    global color_data, upperPt, lowerPt

    os.chdir('/Users/sazamore/BehaviorVids/'+str(ID)+'/vidParam')#/Volumes/ADIPOSE/90/vidParam/')
    filename = str(raw_input('enter global prefix (ID_00_DATE)'))
    filename = filename + '-global.p'
    global_data = pickle.load(open(filename,"rb"))  #import batch-specific data
    #global_data = pickle.load(open('94_00_170329-global.p',"rb"))  #import batch-specific data
    
    #TODO: only reference param_data, get rid of these variables
    upperPt = color_data['upperPt'] = global_data['upperPt']#(390,2) #param_data['upperPt']
    lowerPt = color_data['lowerPt'] = global_data['lowerPt']#(1085,650) #param_data['lowerPt']
    color_data['diameter'] = global_data['diameter']
    return global_data, filename
    
def make_globals(filename):
    global color_data, upperPt, lowerPt, frameCenter
    global_data = {}
    count = 0
    diameter = []
    
    camera.set(cv2.CAP_PROP_POS_FRAMES, 750)
    (grabbed, frame) = camera.read()
    scale = 1300./frame.shape[1]
    frame = cv2.resize(frame,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
    #crop
    if upperPt == [] or lowerPt ==[]:	
        cv2.imshow('Select region of interest',frame)
        cv2.setMouseCallback('Select region of interest', resize_frame)
        cv2.waitKey(8000)
        cv2.destroyWindow('Select region of interest')
    
        #get center
    if frameCenter == [] : 
        cv2.imshow('Click on center',frame)
        cv2.setMouseCallback('Click on center',get_center)
        cv2.waitKey(5000)
        cv2.destroyWindow('Click on center')
        radius = int(633/2) #int(np.mean(diameter)/2)
        cv2.circle(frame,frameCenter,radius,(0,0,0),2)
        color_data['diameter'] = radius*2
    
    #get diam
        while count <= 6:
            cv2.imshow('Click 6 points on drum',frame)
            cv2.setMouseCallback('Click 6 points on drum', get_circ)
            cv2.waitKey(3000)
            cv2.destroyWindow('Click 6 points on drum')
            count += 1
    #print(diameter)
    #pdb.set_trace()
    radius = int(np.mean(color_data['diameter'])/2)
    cv2.circle(frame,frameCenter,radius,(0,0,0),2)
    color_data['diameter'] = radius*2
    
    global_data['diameter']=radius*2
    global_data['upperPt']= upperPt
    global_data['lowerPt']= lowerPt
    global_data['frameCenter'] = frameCenter
    global_data['red_pos']=[]
    global_data['yellow_pos']=[]
    global_data = pickle.dump(global_data,open(filename,"wb"))
    print('Global parameters saved.')
    


In [4]:
#get color position
filepath = '/Users/sazamore/BehaviorVids/94/mp4/'
os.chdir(filepath)#'/Volumes/ADIPOSE/90/Videos/')
vidname = "94_170503_01rpm.mp4"
ID = vidname[:2]
Date = vidname[3:9]
camera = cv2.VideoCapture(vidname)
camera.set(cv2.CAP_PROP_POS_FRAMES, 4500)
(grabbed, frame) = camera.read()

try:
    global_data, filename = import_globals()
    print('Importing global parameters')
except IOError:
    globalname = ID+'_00_'+Date+'-global.p'
    print globalname
    make_globals(globalname)#globalname)#'_00_160811-global.p')
    print('Making global parameters')
print('Upper pt is {}'.format(upperPt))

if len(upperPt)==1:
    upperPt = upperPt[0]
if len(lowerPt)==1:
    lowerPt = lowerPt[0]
    
(grabbed, frame) = camera.read()
scale = 1300./frame.shape[1]
frame = cv2.resize(frame,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
frame = frame[upperPt[1]:lowerPt[1],upperPt[0]:lowerPt[0]]
scaled = True
                          
for color in ['red','yellow']:
    #if color_data[color+'_pos'] == []:
    cv2.imshow(color,frame)			
    cv2.setMouseCallback(color,get_color)
    cv2.waitKey(5000)		#-1 and 0 values seem to hang up. 5000 works.
    cv2.destroyAllWindows()
    
vidname = vidname[:-4] + '_data.p'
pickle.dump(color_data, open(vidname,"wb"))

enter global prefix (ID_00_DATE)94_00_170503
Importing global parameters
Upper pt is (372, 0)
[ 82  51 145]
(105, 431)
[ 80  46 150]
(110, 420)


globalname

In [14]:
x,y = list([383,339])
diameter.extend(np.sqrt((frameCenter[0]-x)**2 + (frameCenter[1]-y)**2))

TypeError: 'numpy.float64' object is not iterable

In [18]:
filepath = '/Users/sazamore/BehaviorVids/94/mp4/'
os.chdir(filepath)#'/Volumes/ADIPOSE/90/Videos/')
vidname = "94_161130_01rpm.mp4"
ID = vidname[:2]
Date = vidname[3:9]
camera = cv2.VideoCapture(vidname)
camera.set(cv2.CAP_PROP_POS_FRAMES, 300)

cv2.destroyAllWindows()
(grabbed, frame) = camera.read()
cv2.imshow('original',frame)
scale = 1300./frame.shape[1]
frame = cv2.resize(frame,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
frame = frame[upperPt[1]:lowerPt[1],upperPt[0]:lowerPt[0]]
# cv2.destroyAllWindows()

# cv2.imshow('trimmed',frame)


In [13]:
cv2.imshow?

In [4]:
upperPt=[363,14]
lowerPt=[1049,663]

In [7]:
# color_data['upperPt'] = upperPt
# color_data['lowerPt'] = lowerPt
color_data

{'diameter': 632,
 'frameCenter': (697, 339),
 'lowerPt': [1050, 675],
 'red_max': (13, 135, 192),
 'red_min': (5, 61, 122),
 'red_pos': (418, 323),
 'refPt': [],
 'upperPt': [355, 15],
 'yellow_max': (22, 182, 199),
 'yellow_min': (9, 67, 110),
 'yellow_pos': (427, 328)}

In [12]:
frameCenter=[]
        #get center
if frameCenter == [] : 
    cv2.imshow('Click on center',frame)
    cv2.setMouseCallback('Click on center',get_center)
    cv2.waitKey(5000)
    print(x,y)
    cv2.destroyWindow('Click on center')
    radius = int(633/2) #int(np.mean(diameter)/2)
    cv2.circle(frame,frameCenter,radius,(0,0,0),2)
    color_data['diameter'] = radius*2

Center is (383, 334)


NameError: name 'x' is not defined

In [37]:
### color_data['yellow_pos'], color_data['red_pos'] 
def get_center(event,x,y,flags,param):			
	"""Finds center of an image using mouse click"""
	global frameCenter, frame	
	if event == cv2.EVENT_LBUTTONDOWN:
		frameCenter = (x,y)
		cv2.circle(frame,(x,y),10,(0,0,0),-1)
		cv2.imshow('Click on center', frame)
		print('Center is '+ str(frameCenter))
		cv2.destroyAllWindows()
		
		color_data["frameCenter"] = frameCenter

In [15]:
#Navigate to video filepath and search for videos by animal number
os.chdir('/Volumes/ADIPOSE/90/')
ID = ["90_170201_10rpm.mp4"]#str(raw_input('Enter animal ID number: ')) + '*.mp4'
#filename = ID
    
#Iterate through videos with animal number. Create and save settings.
for filename in ID:
	#set/reset values
# 	upperPt = []
# 	lowerPt = []
# 	frameCenter = []
	refPt = []
	#diameter = []
	count = 0
	scaled = False
		
	color_data = {
	'red_pos': [],\
	'red_min': [],\
	'red_max': [],\
	'yellow_pos': [],\
	'yellow_min': [],\
	'yellow_max': [],\
	'lowerPt': [],\
	'upperPt': [],\
	'frameCenter': [],\
	'refPt': [],\
	'diameter': [],\
	}
	
	color_set = ['red','yellow']

	#set up camera, grab 200th frame (lighting issues)
	camera = cv2.VideoCapture(ID[0])
	camera.set(cv2.CAP_PROP_POS_FRAMES, 300)
	(grabbed, frame) = camera.read()	

	try:
		import_globals()
	except IOError:
		make_globals()
	print(upperPt)
    
	if grabbed:#scale all videos to same size
		scale = 1300./frame.shape[1]
		frame = cv2.resize(frame,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
		frame = frame[upperPt[1]:lowerPt[1],upperPt[0]:lowerPt[0]]
		scaled = True
	else:
		print "File not read."       

	filename = filename[:-4] + '_data.p'		
		
# 	#crop video
# 	if upperPt == [] or lowerPt ==[]:	
# 		cv2.imshow('Select region of interest',frame)
# 		cv2.setMouseCallback('Select region of interest', resize_frame)
# 		cv2.waitKey(5000)
# 		cv2.destroyWindow('Select region of interest')
			
	#get center
	if frameCenter == [] : 
		cv2.imshow('Click on center',frame)
		cv2.setMouseCallback('Click on center',get_center)
		cv2.waitKey(5000)
		cv2.destroyWindow('Click on center')
        radius = int(633/2)#int(np.mean(diameter))
        cv2.circle(frame,frameCenter,radius,(0,0,0),2)
        color_data['diameter'] = radius*2

	#if diameter == []:
		#get diam
# 		while count < 6:
# 			cv2.imshow('Click 6 points on drum',frame)
# 			cv2.setMouseCallback('Click 6 points on drum', get_circ)
# 			cv2.waitKey(3000)
# 			cv2.destroyWindow('Click 6 points on drum')
# 			count += 1
#         radius = int(np.mean(diameter))
#         cv2.circle(frame,frameCenter,radius,(0,0,0),2)
#         color_data['diameter'] = radius*2
		
	#find reference region
	if refPt ==[]:	
		#get stimulus motion reference
		cv2.imshow('Click pattern for reference',frame)	
		cv2.setMouseCallback('Click pattern for reference',get_reference)
		cv2.waitKey(5000)
		
		#get color values
	for color in color_set:
		if color_data[color+'_pos'] == []:
			cv2.imshow(color,frame)			
			cv2.setMouseCallback(color,get_color)
			cv2.waitKey(5000)		#-1 and 0 values seem to hang up. 5000 works.
			cv2.destroyAllWindows()
        
	cv2.destroyAllWindows()

	pickle.dump(color_data, open(filename,"wb"))
	print(filename)
#		with open(csvfilename,'wb') as csvfile:	#append?
#			#fieldnames = ['upperPt','upperPt','frameCenter_x','frameCenter_y','refPt_x1','refPt_y1','refPt_x2','refPt_y2',color_data.keys()]
#			#writer = csv.DictWriter(csvfile,fieldnames=fieldnames)			
#			#writer.writeheader()
#						
#			writer = csv.writer(csvfile, delimiter = ',')
#			writer.writerow((upperPt[0][0],upperPt[0][1],lowerPt[0][0],lowerPt[0][1], frameCenter[0],frameCenter[1],refPt[0][0],refPt[0][1],refPt[1][0],refPt[1][1]))
#			dicthead = csv.DictWriter(csvfile,color_data.keys())
#			import pdb; pdb.set_trace()
#			#dicthead.writerow(color_data.keys())			
#			for i in color_data.keys():
#				for j in range(len(color_data[i])):
#					dicthead.writerow(color_data[i][j])
#			csvfile.close()
		
		#reset values:
		
			
		#time.sleep(5)

enter global prefix (ID_00_DATE)90_00_170201.p


ValueError: cannot convert float NaN to integer

In [20]:
#frame.shape, upperPt[0][1]

camera.set(cv2.CAP_PROP_POS_FRAMES, 300)
(grabbed, frame) = camera.read()

if grabbed:#scale all videos to same size
    scale = 1300./frame.shape[1]
    frame = cv2.resize(frame,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
    frame = frame[upperPt[1]:lowerPt[1],upperPt[0]:lowerPt[0]]
    scaled = True

while count < 6:
    cv2.imshow('Click 6 points on drum',frame)
    cv2.setMouseCallback('Click 6 points on drum', get_circ)
    cv2.waitKey(3000)
    cv2.destroyWindow('Click 6 points on drum')
    count += 1
#print(diameter)
#pdb.set_trace()
radius = int(np.mean(color_data['diameter'])/2)
cv2.circle(frame,frameCenter,radius,(0,0,0),2)
color_data['diameter'] = radius*2

In [23]:
cv2.destroyWindow('Click 6 points on drum')

In [ ]:
diameter = np.round(np.mean(diameter))
upperPt = global_data['upperPt']
lowerPt = global_data['lowerPt']
diameter = global_data['diameter']

In [17]:
color_data['upperPt'] = upperPt
color_data['lowerPt'] = lowerPt
upperPt

[400, 16]

In [9]:
global_data = {}
global_data['diameter']=diameter
global_data['upperPt']= upperPt[0]
global_data['lowerPt']= lowerPt[0]
global_data['frameCenter'] = frameCenter
global_data['upperPt']
upperPt

[355, 15]

In [ ]:
upperPt = (311,33)#(314, 0)
lowerPt = (977,641)#1069, 658)
frameCenter = (646,360) #(701,322)
diameter = 315

global_data = {}
global_data['diameter']=diameter
global_data['upperPt']= upperPt
global_data['lowerPt']= lowerPt
global_data['frameCenter'] = frameCenter

In [10]:
global_data = pickle.dump(global_data,open('94_00_161130-global.p',"wb"))

In [ ]:
global_data = pickle.load(open('94_00_161130-global.p',"rb"))

In [ ]:
cv2.destroyAllWindows()

In [ ]:
camera.set(cv2.CAP_PROP_POS_FRAMES, 100)
(grabbed, frame) = camera.read()
camera.get(cv2.CAP_PROP_FRAME_COUNT),grabbed

In [ ]:
picklename